# Download all pages for all issues of a given year from Bozner Zeitung

--------------------------install specifically required packages-------------------------------------

    conda install beautifulsoup4
    conda install urllib3

This is an example for how to download files from a complex webpage. This code relies havily on analyzing the structure of the given page, in this case the Tessmann digital archive. Therefore the code only works within this structure. If you want to download files from another webpage or archive, you need to analyze the structure of that page first, insert the different url-string-structures and follow the specific link-structure of the page. The tools and python packages used here are however very versatile, provide many more options and can therefore be used for a large range of webpages.

The code below contains fields marked as 'testloop' and fields marked 'dataloop'. For just downloading the pages of a given year, testloops can be skipped. The testloops are the inner loops of the 'dataloop' fields, i.e. they download or get information about one specific issue. For modifying information and download loops, testloops can be used initially to check whether all changes are meaningful.

In [1]:
import shutil

In [2]:
import re, os

In [3]:
import numpy as np

In [4]:
import urllib3
from bs4 import BeautifulSoup

## First we look at the journal startpage and get a list of available years

it contains the dates for all the years

In [5]:
# this url needs to be provided
start_url = 'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Jahresuebersicht/Zeitung/2'

In [6]:
http = urllib3.PoolManager()

In [7]:
response = http.request('GET', start_url)

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [8]:
soup = BeautifulSoup(response.data)

In [9]:
# find lists in HTML
ylist = soup.find_all('ul')
ylist2 = ylist[2]
#print(ylist2.text)

Now we generate a list with all links to the month overview (starting page) for each year.
For this we need to know the page structure of the month overview, which is inserted in the 'adr_string'-variable.

In [10]:
adr_string = 'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/'
year_link = []
splitlist = ylist2.text.split('\n')
for item in splitlist:
    if item.startswith('1'):
        adr = adr_string+item
        year_link.append(adr)

In [11]:
# generate starting page for each year
year_list = []
for num in range(len(year_link)):
    lsplit = year_link[num].split('/')
    year = lsplit[len(lsplit)-1]
    print (year, year_link[num])
    year_list.append(year)

1842 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1842
1843 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1843
1844 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1844
1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1845
1846 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1846
1847 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1847
1848 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1848
1849 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1849
1850 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1850
1851 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1851
1852 https://digital.tessmann.it/tessman

# Now we select a given year from the list and download the pictures

We could also loop over all the years, but in order to keep things simple we just download one year at the time. The year is selected by the year index below.

In [12]:
# set year index
year_index = 3
print("year = ", year_list[year_index])
month_url =  year_link[year_index]
month_url

year =  1845


'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Monatsuebersicht/Zeitung/2/1845'

In [13]:
# get page data
http = urllib3.PoolManager()
response = http.request('GET', month_url)

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [14]:
# analyze with beautiful soup
soup = BeautifulSoup(response.data)

In [15]:
# you can search for specific tags in page
l1 = soup.find_all('a')

In [16]:
# from the tag list we can get the links to different issues
example = str(l1[20])
example.split('=')

['<a href',
 '"/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/24.01.1845">24</a>']

Now we generate a list with the adresses of all issues.
For generating the complete links the first part of the address needs to be provided as adr_string2.

In [17]:
adr_string2 = 'https://digital.tessmann.it'
datelist = []
linklist = []
for item in l1:
    splititem = str(item).split('=')
    if 'Tagesausgabe' in splititem[1]:
        split2 = splititem[1].split('>')
        split3 = split2[0].split('/')
        split4 = split3[6].split('"')
        datelist.append(split4[0])
        lsplit = split2[0].split(('"'))
        linklist.append(adr_string2+str(lsplit[1]))

In [18]:
# now we have a list of all available issues
for num in range(len(datelist)):
    print(datelist[num], linklist[num])

03.01.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/03.01.1845
10.01.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/10.01.1845
17.01.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/17.01.1845
24.01.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/24.01.1845
31.01.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/31.01.1845
07.02.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/07.02.1845
14.02.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/14.02.1845
21.02.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/21.02.1845
28.02.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Tagesausgabe/Zeitung/2/28.02.1845
07.03.1845 https://digital.tessmann.it/tessmannDigital/Zeitungsa

### Access issue list

We can now get information form the issue list

In [19]:
# testloop; this is for finding the required information, in this case an index number for the download link
num = 0
ind_list = []
issue_url = linklist[num]
http = urllib3.PoolManager()
response = http.request('GET', issue_url)
soup = BeautifulSoup(response.data)
maxpage = 0 # initialize pagenr. to find maximum
for item in soup.find_all('a'):
    adress = item.attrs['href']
    if adress.find('Seite') != -1 :
        #print (adress)
        split = adress.split('/')
        oldmax = maxpage
        newmax = int(split[(len(split)-1)])
        if newmax > oldmax:
            maxpage = newmax
            ind_list.append(adress)
print ('number of pages in this issue: ', maxpage)
print (ind_list)

number of pages in this issue:  6
['/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/1', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/2', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/3', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/4', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/5', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/6']


/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [20]:
# dataloop: we get the information found above for all available dates of this year
ind_list = []
max_list = np.zeros((len(datelist)))
for num in range(len(datelist)):
    ind_list.append([])
    issue_url = linklist[num]
    http = urllib3.PoolManager()
    response = http.request('GET', issue_url)
    soup = BeautifulSoup(response.data)
    maxpage = 0
    for item in soup.find_all('a'):
        adress = item.attrs['href']
        if adress.find('Seite') != -1 :
            split = adress.split('/')
            oldmax = maxpage
            newmax = int(split[(len(split)-1)])
            if newmax > oldmax:
                maxpage = newmax
                ind_list[num].append(adress)
        max_list[num] = maxpage

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

In [21]:
print (max_list)
print (ind_list[5])

[ 6.  6.  4.  6.  8.  8.  8.  8.  6. 11. 11. 11. 10.  8.  6.  6.  7.  8.
 10.  8. 11.  6.  6.  6.  8.  8.  8.  6.  6.  6.  6.  8. 10. 10. 10. 11.
 11. 11.  9.  8.  8.  4.  6.  8.  6.  4.  6.  8.  6.  6.  4.]
['/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/1', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/2', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/3', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/4', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/5', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/6', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/7', '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/07.02.1845/19683/8']


### Sort datelist

We sort the list by date. This is not really required, but makes the download structure easier to understand

In [22]:
# sort datelist
new_list = []
for num, item in enumerate(datelist):
    #print(item.split('.'))
    strl = item.split('.')
    new_list.append([])
    new_list[num].append(str(strl[1])+'-'+str(strl[0])+'-'+str(strl[2]))
    new_list[num].append(item)
    new_list[num].append(ind_list[num])
new_list.sort()
new_list

[['01-03-1845',
  '03.01.1845',
  ['/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/1',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/2',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/3',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/4',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/5',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/6']],
 ['01-10-1845',
  '10.01.1845',
  ['/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/1',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/2',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/3',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/4',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/5',
   '/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/6']],
 ['01-17-1845',
  '17.01.1845',


## Download all pages of each issue

We don't know yet what page we are interested in, therefore we dowload all pages of all issues.

In [23]:
# test address 
num = 0
print(new_list[num][0])
adstr1 = 'https://digital.tessmann.it'
adstr2 = str(new_list[num][2][0])
page_url = adstr1+adstr2
page_url

01-03-1845


'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/1'

In [24]:
# loop over issues for this year
page_url_list = []
for num in range(len(new_list)):
    page_url_list.append([])
    for p in range(len(new_list[num][2])):
        adstr2 = str(new_list[num][2][p])
        page_url_list[num].append(adstr1+adstr2)

In [25]:
# list of links with all pages for all issues in this year
page_url_list

[['https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/1',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/2',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/3',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/4',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/5',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/03.01.1845/19678/6'],
 ['https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/1',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/2',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845/19679/3',
  'https://digital.tessmann.it/tessmannDigital/Zeitungsarchiv/Seite/Zeitung/2/1/10.01.1845

In [26]:
# from these links, we get a list of the picture links
pictures = []
for num in range(len(page_url_list)):
    pictures.append([])
    for p in range(len(page_url_list[num])):
        http = urllib3.PoolManager()
        response = http.request('GET', page_url_list[num][p])
        soup = BeautifulSoup(response.data)
        for item in soup.find_all('a'):
            adress = item.attrs['href']
            if adress.find('media/image') != -1 :
                print (num, adress)
                pictures[num].append(adress)

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0 https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_1_object_459681.png?auth=32ba81b0586768bd4acffca31de634a4


/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


0 https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_2_object_459683.png?auth=a7592f24b7b2efda8aad9f154d40295c


/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

50 https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/26_12_1845/BZZ_1845_12_26_2_object_458388.png?auth=87b5054332f04b1d569c45272d892827


/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


50 https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/26_12_1845/BZZ_1845_12_26_3_object_458391.png?auth=00e5e9c3194bae42e98774f21b3b51fc


/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


50 https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/26_12_1845/BZZ_1845_12_26_4_object_458394.png?auth=b7fe5750e2fc1727b182c4be5e152c6b


In [27]:
pictures

[['https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_1_object_459681.png?auth=32ba81b0586768bd4acffca31de634a4',
  'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_2_object_459683.png?auth=a7592f24b7b2efda8aad9f154d40295c',
  'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_3_object_459686.png?auth=3b7af8e547106266bfbf39779907d00c',
  'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_4_object_459688.png?auth=e9647f33f2df1a9696bab4c5b2093d3d',
  'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_5_object_459690.png?auth=ff954a098abe00068e791b7348bf2dd3',
  'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_6_object_459693.png?auth=d811da803e8cadbc113397e79f310be9'],
 ['https://digital.tessmann.it/mediaArchive/media/image/Page/BZ

Now we have all the download links. We create a folder for the given year and all pages of all issues into a subfolder labelled with the corresponding date.

In [28]:
# make folder
folder = 'wy_BZ_'+str(year_list[year_index])
if not os.path.exists(folder):
    os.mkdir(folder)

In [29]:
# show list
for num in range(len(datelist)):
    print (new_list[num][1], pictures[num])

03.01.1845 ['https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_1_object_459681.png?auth=32ba81b0586768bd4acffca31de634a4', 'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_2_object_459683.png?auth=a7592f24b7b2efda8aad9f154d40295c', 'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_3_object_459686.png?auth=3b7af8e547106266bfbf39779907d00c', 'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_4_object_459688.png?auth=e9647f33f2df1a9696bab4c5b2093d3d', 'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_5_object_459690.png?auth=ff954a098abe00068e791b7348bf2dd3', 'https://digital.tessmann.it/mediaArchive/media/image/Page/BZZ/1845/03_01_1845/BZZ_1845_01_03_6_object_459693.png?auth=d811da803e8cadbc113397e79f310be9']
10.01.1845 ['https://digital.tessmann.it/mediaArchive/media/imag

In [30]:
### testblock; test downloading for a single issue here!!!!
num = 0
subfolder = os.path.join(folder,str(new_list[num][0]))
if not os.path.exists(subfolder):
    os.mkdir(subfolder)
for p in range(len(new_list[num][2])):
    filename = os.path.join(subfolder,(str(new_list[num][0])+'_'+str(p)+'.png'))
    http = urllib3.PoolManager()
    c = urllib3.PoolManager()

    with c.request('GET',pictures[num][p], preload_content=False) as resp, open(filename, 'wb') as out_file:
        shutil.copyfileobj(resp, out_file)

    resp.release_conn()  

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

In [31]:
## Download loop: here we download all the files for the selected year
for num in range(len(new_list)):
    subfolder = os.path.join(folder,str(new_list[num][0]))
    if not os.path.exists(subfolder):
        os.mkdir(subfolder)
    for p in range(len(new_list[num][2])):
        filename = os.path.join(subfolder,(str(new_list[num][0])+'_'+str(p)+'.png'))     
        http = urllib3.PoolManager()
        c = urllib3.PoolManager()
        with c.request('GET',pictures[num][p], preload_content=False) as resp, open(filename, 'wb') as out_file:
            shutil.copyfileobj(resp, out_file)

        resp.release_conn()  

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

...

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond

/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/miniconda3/envs/AutoSearch/lib/python3.7/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/nplattner/minicond